<a href="https://colab.research.google.com/github/carlosvalenciano/Project4/blob/main/Stock_Spark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.0'
spark_version = 'spark-3.5.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:6 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:7 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,036 kB]
Get:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [1,578 kB]
Get:13 https://ppa.launchpadcontent.net/de

In [21]:
# Import packages
from pyspark.sql import SparkSession
import time
import datetime as dt

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [57]:
# 1. Read in the csv into a DataFrame.

df = spark.read.format("csv").option("header", "true").load("Multiple_Stocks.csv")
df.show()


+----------+-----------+-----------+-----------+-----------+-----------+---------+------------+------------+---------------+
|      Date|       Open|       High|        Low|      Close|  Adj_Close|   Volume|company_name|Scaled_Close|predicted_close|
+----------+-----------+-----------+-----------+-----------+-----------+---------+------------+------------+---------------+
|2019-12-12|66.94499969|68.13999939|66.83000183|67.86499786|66.11624908|137310400|       APPLE| 0.045575656|    72.47750092|
|2019-12-13|67.86499786|68.82499695|67.73249817|68.78749847|67.01499176|133587600|       APPLE| 0.048372223|    72.44999695|
|2019-12-16|      69.25|70.19750214|69.24500275|69.96499634|68.16214752|128186000|       APPLE| 0.051941815|    72.87999725|
|2019-12-17|69.89250183|70.44249725|69.69999695|70.10250092|68.29610443|114158400|       APPLE| 0.052358661|    73.41249847|
|2019-12-18|69.94999695|70.47499847|69.77999878|69.93499756|68.13290405|116028400|       APPLE| 0.051850873|    75.08750153|


In [58]:
# 2. Create a temporary view of the DataFrame.
df.createOrReplaceTempView('stocks')


In [85]:
# 3.
spark.sql("""
SELECT
  company_name AS Company,
  (Avg(predicted_close)) AS Predicted_Close
  FROM stocks
  WHERE Date between ('2023-12-07') and ('2023-12-11')
  GROUP BY Company
  ORDER BY Predicted_Close DESC
""").show()

+---------+------------------+
|  Company|   Predicted_Close|
+---------+------------------+
|MICROSOFT|       373.3289693|
|     META| 328.2066650333333|
|    APPLE|194.51652016666665|
|   AMAZON|       145.0767466|
|   GOOGLE|133.76257833333332|
+---------+------------------+



In [87]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
spark.sql("""
SELECT
  company_name AS Company,
  Close AS Close
  FROM stocks
  WHERE Date ('2023-12-07') and ('2023-12-11')
  GROUP BY Company
  ORDER BY Predicted_Close DESC
""").show()


AnalysisException: ignored